In [245]:
import pandas as pd
import numpy as np
import sklearn
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, timedelta
from matplotlib.pyplot import cm
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support,roc_auc_score,balanced_accuracy_score,precision_recall_curve,auc
%matplotlib inline


In [225]:
# ACLED
path = '~/data/ACLED/country-jul23/2017-01-01-2021-07-23-Syria.csv'
# path = '~/data/ACLED/country-jul23/2015-01-01-2021-07-23-Yemen.csv'
path = '~/data/ACLED/country-jul23/2017-01-01-2021-07-23-Afghanistan.csv'

country_name = path.split('-')[7][:-4]
filename = path.split('/')[-1]
print('path',path,'filename',filename,country_name,'country_name')
start_year = int(filename.split('-')[0])
start_month = int(filename.split('-')[1])
start_day = int(filename.split('-')[2])
end_year = int(filename.split('-')[3])
end_month = int(filename.split('-')[4])
end_day = int(filename.split('-')[5])

df = pd.read_csv(path,sep=';')
df = df.drop_duplicates(subset=['data_id'], keep='first')
df['event_date'] = pd.to_datetime(df['event_date'])
print(df.columns)
df.sort_values(by=['event_date'],inplace=True ) 


path ~/data/ACLED/country-jul23/2017-01-01-2021-07-23-Afghanistan.csv filename 2017-01-01-2021-07-23-Afghanistan.csv Afghanistan country_name
Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')


In [226]:
event_type_column = 'sub_event_type'
# event_type_column = 'event_type'

subevents = df[event_type_column].unique()
print(len(subevents),subevents)
delta_value = 1
if delta_value == 1:
    level = 'day'
elif delta_value == 7:
    level = 'week'
elif delta_value == 14:
    level = 'biweek'
elif delta_value == 30:
    level = 'month'

subevent_count_dict = {}
start_date = date(start_year, start_month, start_day)
end_date = date(end_year, end_month, end_day)
delta = timedelta(days=delta_value)
n_days = 0
last_date = start_date - delta
while start_date <= end_date:
    last_date = start_date
    start_date += delta
    n_days += 1
print('n_days =',n_days)
for v in subevents:
    subevent_count_dict[v] = np.array([0 for i in range(n_days)])


24 ['Armed clash' 'Remote explosive/landmine/IED'
 'Non-state actor overtakes territory' 'Shelling/artillery/missile attack'
 'Attack' 'Air/drone strike' 'Peaceful protest'
 'Abduction/forced disappearance' 'Government regains territory'
 'Disrupted weapons use' 'Change to group/activity' 'Mob violence'
 'Suicide bomb' 'Non-violent transfer of territory'
 'Protest with intervention' 'Looting/property destruction' 'Arrests'
 'Sexual violence' 'Headquarters or base established' 'Grenade'
 'Violent demonstration' 'Agreement' 'Excessive force against protesters'
 'Other']
n_days = 1665


In [227]:
start_date = date(start_year, start_month, start_day)
end_date = date(end_year, end_month, end_day)
delta = timedelta(days=delta_value)
day_i = 0
last_date = start_date - delta
while start_date <= end_date:
    last_date_str = last_date.strftime("%Y-%m-%d") #("%d %B %Y")
    date_str = start_date.strftime("%Y-%m-%d")
    df_day = df.loc[(df['event_date'] > last_date_str) & (df['event_date'] <= date_str)]
    if day_i%300==0:
        print('#',day_i,len(df_day),len(df))
    df_count = df_day[event_type_column].value_counts().rename_axis('unique_values').reset_index(name='counts')
    for i,row in df_count.iterrows():
        subevent_count_dict[row['unique_values']][day_i] = row['counts']
    last_date = start_date
    start_date += delta
    day_i += 1
print('day_i =',day_i)

# 0 46 55854
# 300 39 55854
# 600 27 55854
# 900 41 55854
# 1200 14 55854
# 1500 33 55854
day_i = 1665


In [228]:
def movingaverage(a, n=3) :
    padding = []
    for i in range(n-1):
        padding.append(a[:i+1].mean())
    padding = np.array(padding)
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return np.concatenate((padding, ret[n - 1:] / n),0)

In [229]:
event_set_protest = ['Protest with intervention','Excessive force against protesters','Peaceful protest']
subevent_count_dict['Protests'] = subevent_count_dict['Protest with intervention'] + subevent_count_dict['Peaceful protest'] + subevent_count_dict['Excessive force against protesters']
# del subevent_count_dict['Protest with intervention']
# del subevent_count_dict['Excessive force against protesters']
# del subevent_count_dict['Peaceful protest']

subevent_count_dict

{'Armed clash': array([21, 33, 21, ...,  2, 13,  2]),
 'Remote explosive/landmine/IED': array([13,  9,  4, ...,  1,  0,  0]),
 'Non-state actor overtakes territory': array([1, 2, 2, ..., 0, 0, 0]),
 'Shelling/artillery/missile attack': array([4, 3, 0, ..., 2, 0, 1]),
 'Attack': array([2, 1, 3, ..., 0, 1, 0]),
 'Air/drone strike': array([4, 6, 1, ..., 3, 4, 0]),
 'Peaceful protest': array([1, 0, 1, ..., 1, 0, 0]),
 'Abduction/forced disappearance': array([0, 1, 0, ..., 0, 0, 0]),
 'Government regains territory': array([0, 1, 0, ..., 0, 0, 2]),
 'Disrupted weapons use': array([0, 1, 0, ..., 0, 0, 0]),
 'Change to group/activity': array([0, 0, 1, ..., 0, 0, 0]),
 'Mob violence': array([0, 0, 1, ..., 0, 0, 0]),
 'Suicide bomb': array([0, 0, 0, ..., 0, 0, 0]),
 'Non-violent transfer of territory': array([0, 0, 0, ..., 0, 2, 0]),
 'Protest with intervention': array([0, 0, 0, ..., 0, 0, 0]),
 'Looting/property destruction': array([0, 0, 0, ..., 0, 0, 0]),
 'Arrests': array([0, 0, 0, ..., 0, 0

In [230]:
subevent_count_dict.keys()

dict_keys(['Armed clash', 'Remote explosive/landmine/IED', 'Non-state actor overtakes territory', 'Shelling/artillery/missile attack', 'Attack', 'Air/drone strike', 'Peaceful protest', 'Abduction/forced disappearance', 'Government regains territory', 'Disrupted weapons use', 'Change to group/activity', 'Mob violence', 'Suicide bomb', 'Non-violent transfer of territory', 'Protest with intervention', 'Looting/property destruction', 'Arrests', 'Sexual violence', 'Headquarters or base established', 'Grenade', 'Violent demonstration', 'Agreement', 'Excessive force against protesters', 'Other', 'Protests'])

In [231]:
SUBEVENTS = ['Abduction/forced disappearance', 'Agreement', 'Air/drone strike',
       'Armed clash', 'Arrests', 'Attack', 'Change to group/activity',
       'Chemical weapon', 'Disrupted weapons use',
       'Excessive force against protesters',
       'Government regains territory', 'Grenade',
       'Headquarters or base established', 'Looting/property destruction',
       'Mob violence', 'Non-state actor overtakes territory',
       'Non-violent transfer of territory', 'Other', 'Peaceful protest',
       'Protest with intervention', 'Remote explosive/landmine/IED',
       'Sexual violence', 'Shelling/artillery/missile attack',
       'Suicide bomb', 'Violent demonstration']

In [237]:
# subevent_count_dict['Chemical weapon']

KeyError: 'Chemical weapon'

In [238]:
# build sequence data
X = []
for k in SUBEVENTS:
    try:
        v = subevent_count_dict[k].tolist()
        X.append(v)
    except:
        pass
    
X = np.array(X)
X = np.swapaxes(X,0,1)
Y = subevent_count_dict['Protests']
X.shape,Y.shape

((1665, 24), (1665,))

In [239]:
window = 14
horizon = 1
pred_window = 3
ii = 0
data_X = []
data_Y = []
for i in range(0,len(X),horizon+pred_window-1): # no overlap of pre_window
# for i in range(0,len(X),horizon): # overlap 1
#     print('x',i,i+window,' y',i+window,i+window+pred_window)
    data_X.append(X[i:i+window])
    protest = Y[i+window:i+window+pred_window].sum()
#     print(Y[i+window:i+window+pred_window])
#     print(X[i:i+window],Y[i+window:i+window+pred_window-1])
    data_Y.append(1 if protest > 0 else 0)
    if i+window >=len(X) or i+window+pred_window-1 >= len(X):
        break
    ii+=1
print(ii)
#     pass

data_X = np.array(data_X)
data_Y = np.array(data_Y)
data_X.shape,data_Y.shape,data_Y.mean()

550


((551, 14, 24), (551,), 0.5245009074410163)

In [241]:
flat_data_X = data_X.reshape(data_X.shape[0],-1)
X_train, X_test, y_train, y_test = train_test_split(flat_data_X, data_Y, stratify=data_Y, test_size=0.25,
                                                    shuffle = True,
                                                    random_state=42)

clf = LogisticRegression(random_state=42,max_iter=5000).fit(X_train, y_train)
acc = clf.score(X_test, y_test)
print('acc',acc)
y_pred = clf.predict(X_test)
bacc = balanced_accuracy_score(y_test, y_pred)
print('bacc',bacc)
y_prob = clf.predict_proba(X_test)[:,1]

precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
# print(precision, recall, thresholds )
area = auc(recall, precision)
print("Area Under PR Curve(AP)", area)  #should be same as AP?


aucv = roc_auc_score(y_test, y_prob)
print('auc',aucv)
precision_recall_fscore_support(y_test, y_pred, average='binary')

acc 0.5942028985507246
bacc 0.5921717171717171
Area Under PR Curve(AP) 0.6579023227546095
auc 0.6292087542087541


(0.6052631578947368, 0.6388888888888888, 0.6216216216216216, None)

In [242]:
clf = MLPClassifier(random_state=42, max_iter=500).fit(X_train, y_train)

acc = clf.score(X_test, y_test)
print('acc',acc)
y_pred = clf.predict(X_test)
bacc = balanced_accuracy_score(y_test, y_pred)
print('bacc',bacc)
y_prob = clf.predict_proba(X_test)[:,1]

precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
# print(precision, recall, thresholds )
area = auc(recall, precision)
print("Area Under PR Curve(AP)", area)  #should be same as AP?


aucv = roc_auc_score(y_test, y_prob)
print('auc',aucv)
precision_recall_fscore_support(y_test, y_pred, average='binary')

acc 0.5579710144927537
bacc 0.5555555555555556
Area Under PR Curve(AP) 0.642544362443275
auc 0.6148989898989898


(0.5714285714285714, 0.6111111111111112, 0.5906040268456376, None)

In [243]:
clf = svm.SVC(probability=True)
clf.fit(X_train, y_train)
acc = clf.score(X_test, y_test)
print('acc',acc)
y_pred = clf.predict(X_test)

bacc = balanced_accuracy_score(y_test, y_pred)
print('bacc',bacc)

y_prob = clf.predict_proba(X_test)[:,1]

precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
# print(precision, recall, thresholds )
area = auc(recall, precision)
print("Area Under PR Curve(AP)", area)  #should be same as AP?


aucv = roc_auc_score(y_test, y_prob)
print('auc',aucv)
precision_recall_fscore_support(y_test, y_pred, average='binary')

acc 0.5507246376811594
bacc 0.5473484848484849
Area Under PR Curve(AP) 0.592258143487848
auc 0.5591329966329966


(0.5625, 0.625, 0.5921052631578947, None)

In [171]:
# y_prob

In [193]:
# del auc

In [271]:
mydir = '/home/sdeng/workspace/gdelt_data_preprocess/event/'
# file_list = glob.glob(mydir + "*NI*.json")
# file_list = glob.glob(mydir + "*CA*.json")
# country_name = 'EG'
country_name = 'NI'

file_list = glob.glob(mydir + "*{}*.json".format(country_name))
file_list
df_list = []
for f in file_list:
    cur_df = pd.read_json(f,lines=True)
    df_list.append(cur_df)
#     print(cur_df.head())
#     break
    
df = pd.concat(df_list, ignore_index=True)
df['event_date'] = pd.to_datetime(df['event_date'],format='%Y%m%d' )
df.sort_values(by=['event_date'],inplace=True) 
df = df.loc[df['IsRootEvent'] == 1]

df['event_date'] = df.event_date.dt.strftime('%Y-%m-%d')
df = df.loc[df['event_date']>='2015-01-01']

In [272]:
def getRoot(x):
    x = int(x)
    if len(str(x)) == 4: # 1128
        return x // 100
    elif len(str(x)) == 3:
        if x // 10 < 20: # 190
            return x // 10
        else:
            return x // 100
    else:
        return x // 10
    
def movingaverage(a, n=3) :
    padding = []
    for i in range(n-1):
        padding.append(a[:i+1].mean())
    padding = np.array(padding)
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return np.concatenate((padding, ret[n - 1:] / n),0)

df = df.loc[df['EventCode'] != '---'] 
df['RootEventCode'] = df['EventCode'].apply(lambda x: getRoot(x) )
 

In [273]:
start_year = 2015
start_month = 1
start_day = 1
end_year = 2020
end_month = 12
end_day = 31
event_type_column = 'EventCode'
event_type_column = 'RootEventCode'
delta_value = 1
if delta_value == 1:
    level = 'day'
elif delta_value == 7:
    level = 'week'
elif delta_value == 14:
    level = 'biweek'
elif delta_value == 30:
    level = 'month'
subevents = df[event_type_column].unique()
print(len(subevents),subevents)
subevent_count_dict = {}
start_date = date(start_year, start_month, start_day)
end_date = date(end_year, end_month, end_day)
delta = timedelta(days=delta_value)
n_days = 0
last_date = start_date - delta
while start_date <= end_date:
#     print('last_date',last_date,'start_date',start_date )
    last_date = start_date
    start_date += delta
    n_days += 1
print('n_days =',n_days)
# print('n_days =',len(df['event_date'].unique()))
for v in subevents:
    subevent_count_dict[v] = np.array([0 for i in range(n_days)])


19 [ 4  9  1  3  2 18  8 17  7 19 11  6 12  5 10 13 15 16 14]
n_days = 2192


In [274]:
# for loop day.... save count of each subevent.
start_date = date(start_year, start_month, start_day)
end_date = date(end_year, end_month, end_day)
delta = timedelta(days=delta_value)
day_i = 0
last_date = start_date - delta
# print('last_date',last_date,'start_date',start_date,'end_date',end_date)

while start_date <= end_date:
#     print('last_date',last_date,'start_date',start_date )
    last_date_str = last_date.strftime("%Y-%m-%d") #("%d %B %Y")
    date_str = start_date.strftime("%Y-%m-%d")
#     print('last_date_str',last_date_str,' --- date_str',date_str)
    df_day = df.loc[(df['event_date'] > last_date_str) & (df['event_date'] <= date_str)]
    if day_i%200==0:
        print('#',len(df_day),len(df))
#         print(df_day['sub_event_type'] )
    df_count = df_day[event_type_column].value_counts().rename_axis('unique_values').reset_index(name='counts')
#     print('df_count',df_count,df)
    for i,row in df_count.iterrows():
        subevent_count_dict[row['unique_values']][day_i] = row['counts']
    last_date = start_date
    start_date += delta
    day_i += 1
print('day_i =',day_i)

# 7 2915385
# 1938 2915385
# 1691 2915385
# 2160 2915385
# 1214 2915385
# 1686 2915385
# 931 2915385
# 1543 2915385
# 1423 2915385
# 1274 2915385
# 1362 2915385
day_i = 2192


In [286]:
subevent_count_dict
SUBEVENTS = [i+1 for i in range(20)]
SUBEVENTS

# build sequence data
X = []
for k in SUBEVENTS:
    try:
        v = subevent_count_dict[k].tolist()
        X.append(v)
    except:
        pass
    
X = np.array(X)
X = np.swapaxes(X,0,1)
Y = subevent_count_dict[14]
y_threshod = np.percentile(Y, 95)#Y.mean()

X.shape,Y.shape, y_threshod

((2192, 19), (2192,), 30.0)

In [287]:
window = 14
horizon = 1
pred_window = 3
ii = 0
data_X = []
data_Y = []
for i in range(0,len(X),horizon+pred_window-1): # no overlap of pre_window
# for i in range(0,len(X),horizon): # overlap 1
#     print('x',i,i+window,' y',i+window,i+window+pred_window)
    data_X.append(X[i:i+window])
    protest = Y[i+window:i+window+pred_window].sum()
#     print(Y[i+window:i+window+pred_window])
#     print(X[i:i+window],Y[i+window:i+window+pred_window-1])
    data_Y.append(1 if protest > y_threshod else 0)
    if i+window >=len(X) or i+window+pred_window-1 >= len(X):
        break
    ii+=1
print(ii)
#     pass

data_X = np.array(data_X)
data_Y = np.array(data_Y)
data_X.shape,data_Y.shape,data_Y.mean()

726


((727, 14, 19), (727,), 0.7455295735900963)

In [288]:
flat_data_X = data_X.reshape(data_X.shape[0],-1)
X_train, X_test, y_train, y_test = train_test_split(flat_data_X, data_Y, stratify=data_Y, test_size=0.25,
                                                    shuffle = True,
                                                    random_state=42)

clf = LogisticRegression(random_state=42,max_iter=5000).fit(X_train, y_train)
acc = clf.score(X_test, y_test)
print('acc',acc)
y_pred = clf.predict(X_test)
bacc = balanced_accuracy_score(y_test, y_pred)
print('bacc',bacc)
y_prob = clf.predict_proba(X_test)[:,1]

precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
# print(precision, recall, thresholds )
area = auc(recall, precision)
print("Area Under PR Curve(AP)", area)  #should be same as AP?


aucv = roc_auc_score(y_test, y_prob)
print('auc',aucv)
precision_recall_fscore_support(y_test, y_pred, average='binary')

acc 0.6483516483516484
bacc 0.6064578005115089
Area Under PR Curve(AP) 0.8675722643403999
auc 0.6663203324808183


/home/sdeng/anaconda2/envs/jupyter/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(0.8103448275862069, 0.6911764705882353, 0.746031746031746, None)

In [278]:
clf = svm.SVC(probability=True)
clf.fit(X_train, y_train)
acc = clf.score(X_test, y_test)
print('acc',acc)
y_pred = clf.predict(X_test)

bacc = balanced_accuracy_score(y_test, y_pred)
print('bacc',bacc)

y_prob = clf.predict_proba(X_test)[:,1]

precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
# print(precision, recall, thresholds )
area = auc(recall, precision)
print("Area Under PR Curve(AP)", area)  #should be same as AP?


aucv = roc_auc_score(y_test, y_prob)
print('auc',aucv)
precision_recall_fscore_support(y_test, y_pred, average='binary')

acc 0.8241758241758241
bacc 0.5294117647058824
Area Under PR Curve(AP) 0.927138454382989
auc 0.7927265500794913


(0.8222222222222222, 1.0, 0.9024390243902439, None)

In [279]:
clf = MLPClassifier(random_state=42, max_iter=500).fit(X_train, y_train)

acc = clf.score(X_test, y_test)
print('acc',acc)
y_pred = clf.predict(X_test)
bacc = balanced_accuracy_score(y_test, y_pred)
print('bacc',bacc)
y_prob = clf.predict_proba(X_test)[:,1]

precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
# print(precision, recall, thresholds )
area = auc(recall, precision)
print("Area Under PR Curve(AP)", area)  #should be same as AP?


aucv = roc_auc_score(y_test, y_prob)
print('auc',aucv)
precision_recall_fscore_support(y_test, y_pred, average='binary')

acc 0.7417582417582418
bacc 0.6033386327503975
Area Under PR Curve(AP) 0.8980921749982628
auc 0.6708068362480127


(0.8531468531468531, 0.8243243243243243, 0.8384879725085911, None)